In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

# ============================================================
# 1. DANH SÁCH FILE KẾT QUẢ TỪ CÁC SEGMENT
# ============================================================
# Giả sử bạn có 2 segments, thêm tên file vào list này
segment_files = [
    r'C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg1_result.parquet',
    r"C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg2_result.parquet",
    r"C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg3_result.parquet" 
]

# ============================================================
# 2. ĐỌC VÀ GỘP DỮ LIỆU (STACKING)
# ============================================================
df_list = []
for f in segment_files:
    try:
        temp_df = pd.read_parquet(f)
        df_list.append(temp_df)
        print(f"-> Đã đọc: {f} | Shape: {temp_df.shape}")
    except Exception as e:
        print(f"Lỗi không đọc được file {f}: {e}")

# Gộp lại thành 1 DataFrame duy nhất
df_total = pd.concat(df_list, axis=0, ignore_index=True)
print(f"\n=> TỔNG SỐ LƯỢNG MẪU (ALL SEGMENTS): {df_total.shape}")

# ============================================================
# 3. HÀM TÍNH GINI
# ============================================================
def calculate_gini(y_true, y_prob):
    try:
        # Gini = 2 * AUC - 1
        auc = roc_auc_score(y_true, y_prob)
        gini = 2 * auc - 1
        return gini
    except:
        return np.nan

# ============================================================
# 4. TÍNH GINI CHUNG (OVERALL) THEO TỪNG TẬP DỮ LIỆU
# ============================================================
print("\n" + "="*40)
print("   KẾT QUẢ GINI CHUNG (ALL SEGMENTS)")
print("="*40)

# Duyệt qua từng loại tập dữ liệu
summary_gini = []

for dtype in ['TRAIN', 'OOS', 'OOT']:
    # Lọc dữ liệu theo loại (Train/OOS/OOT) của TOÀN BỘ các segment gộp lại
    subset = df_total[df_total['DATA_TYPE'] == dtype]
    
    if len(subset) > 0:
        # Tính Gini
        gini_score = calculate_gini(subset['BAD_NEXT_12M'], subset['PREDICTED_PROB'])
        
        print(f"Dataset: {dtype:<10} | Records: {len(subset):<7} | Gini: {gini_score:.4f}")
        
        summary_gini.append({
            'DATA_TYPE': dtype,
            'Records': len(subset),
            'Overall_Gini': gini_score
        })
    else:
        print(f"Dataset: {dtype:<10} | Không có dữ liệu")

# ============================================================
# 5. (TÙY CHỌN) KIỂM TRA GINI CHI TIẾT LẠI TỪNG SEGMENT
# ============================================================
# Bước này để đảm bảo khi gộp vào không bị sai sót so với lúc chạy lẻ
print("\n" + "="*40)
print("   BREAKDOWN THEO TỪNG SEGMENT")
print("="*40)

pivot_gini = df_total.groupby(['SEGMENT', 'DATA_TYPE']).apply(
    lambda x: calculate_gini(x['BAD_NEXT_12M'], x['PREDICTED_PROB'])
).reset_index(name='GINI')

# Pivot table cho dễ nhìn
report_table = pivot_gini.pivot(index='SEGMENT', columns='DATA_TYPE', values='GINI')
print(report_table)

# ============================================================
# 6. XUẤT KẾT QUẢ CUỐI CÙNG (MASTER FILE)
# ============================================================
# File này dùng để đẩy sang hệ thống chấm điểm hoặc làm báo cáo tổng
df_total.to_parquet("FINAL_MODEL_SCORED_ALL.parquet", index=False)

-> Đã đọc: C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg1_result.parquet | Shape: (787973, 5)
-> Đã đọc: C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg2_result.parquet | Shape: (296611, 5)
-> Đã đọc: C:\Users\PC\Documents\GitHub\Khoa-luan\Log reg\seg3_result.parquet | Shape: (655653, 5)

=> TỔNG SỐ LƯỢNG MẪU (ALL SEGMENTS): (1740237, 5)

   KẾT QUẢ GINI CHUNG (ALL SEGMENTS)
Dataset: TRAIN      | Records: 1137807 | Gini: 0.6702
Dataset: OOS        | Records: 300317  | Gini: 0.6875
Dataset: OOT        | Records: 302113  | Gini: 0.6863

   BREAKDOWN THEO TỪNG SEGMENT


C:\Users\PC\AppData\Local\Temp\ipykernel_15376\1668024396.py:79: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pivot_gini = df_total.groupby(['SEGMENT', 'DATA_TYPE']).apply(


DATA_TYPE                       OOS       OOT     TRAIN
SEGMENT                                                
1. Prime (High Liquidity)  0.692659  0.692551  0.690820
2. Standard (Low DTI)      0.483297  0.469779  0.468570
3. Subprime (High Risk)    0.456057  0.446782  0.451203
